### Intro

This note book will organize the galaxy catalogs from the UniverseMachine model using the small multidark planck simulation (SMDPL).  The results are provided by Peter Behroozi, and the initial catalogs are made by Christopher Bradshaw. 

The models are for galaxies at z~0.4.  Two models are provided: 
1. `sfr_catalog_0.712400.txt.gz`: old fashion `bcg` and `icl` components one. 
2. `sfr_catalog_insitu_exsitu_0.712400.txt.gz`: new `in-situ` and `ex-situ` components one. 
3. `particles_0.712400.txt.gz`: DM particles file. 

The useful information has been summarized into a smaller catalog, also the central and satellite galaxies are separated now.  (`.npz`) format. 

Here, we: 

1. Explore the basic properties of these new models. 
2. Compare their SMF with observed ones. 
3. Explore new M100-M10 models.
4. Select the relevant massive galaxies, and save the value-added catalog. 

We also need to downsample the particle tables and pre-compute the `DeltaSigma` signals around the selected massive galaxies.

#### Small MultiDark Simulation

* For details, please see: https://www.cosmosim.org/cms/simulations/smdpl/

![](cosmosim_smdpl_sum.png)

In [11]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

import os
import sys
import copy 
from time import time

import numpy as np 
import matplotlib.pyplot as plt
    
import matplotlib.mlab as ml
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

%config InlineBackend.figure_format='retina'
plt.rcParams['figure.dpi'] = 100.0
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12.0
plt.rc('text', usetex=True)

# Personal repo
from stellar_mass_function import compute_smf, \
    bootstrap_smf
from load_mdpl2_mock import value_added_smdpl_mock
from model_predictions import total_stellar_mass_including_satellites
    
from hsc_massive import \
    s16a_path, \
    sample_selection, \
    prepare_sed, \
    catalog_summary, \
    smhm, \
    plotting

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from astropy.table import Table, Column, vstack

# Summary catalogs for SMDPL-UniverseMachine model

In [4]:
# Setup environment 
envir = s16a_path.set_env(version='astro5')
#envir = s16a_path.set_env(version='kungpao')

# Folder for new UM model 
um_dir = os.path.join(envir['s16a'], 'um2/um2_new')
um_smdpl_dir = os.path.join(um_dir, 'um_smdpl_small')

# Summary catalogs for new UM models 
um_smdpl_old = np.load(os.path.join(um_smdpl_dir, 
                                    'sfr_catalog_0.712400_final.npz'))
um_smdpl_new = np.load(os.path.join(um_smdpl_dir, 
                                    'sfr_catalog_insitu_exsitu_0.712400_final.npz'))

# Results for central and satellite galaxies
um_smdpl_old_cen = Table(um_smdpl_old['centrals'])
um_smdpl_old_sat = Table(um_smdpl_old['satellites'])

um_smdpl_new_cen = Table(um_smdpl_new['centrals'])
um_smdpl_new_sat = Table(um_smdpl_new['satellites'])

print(len(um_smdpl_old_cen), len(um_smdpl_old_sat))
print(len(um_smdpl_new_cen), len(um_smdpl_new_sat))

print(um_smdpl_old_cen.colnames)

(385125, 10809469)
(385125, 10809469)
['id', 'upid', 'x', 'y', 'z', 'm', 'mp', 'sm', 'icl', 'sfr']


In [30]:
# vstack the centrals and satellites 

um_smdpl_old_all = vstack([um_smdpl_old_cen, um_smdpl_old_sat])
um_smdpl_new_all = vstack([um_smdpl_new_cen, um_smdpl_new_sat])

# Value added the catalogs 
um_smdpl_old_all = value_added_smdpl_mock(um_smdpl_old_all)
um_smdpl_new_all = value_added_smdpl_mock(um_smdpl_new_all)

In [29]:
def umPrepCatalog(um_mock, um_min_mvir=None, use_mvir=False):
    """
    Prepare the UniverseMachine mock catalog.

    The goal is to prepare a FITS catalog that include all
    necessary information.
    During the modeling part, we just need to load this catalog once.
    """
    # Sort the catalog based on the host halo ID
    um_mock.sort('halo_hostid')

    # Make a mask for central galaxies
    mask_central = um_mock['upid'] == -1
    um_mock.add_column(Column(data=mask_central,
                              name='mask_central'))

    # Add a column as the BCG+ICL mass
    um_mock.add_column(Column(data=(um_mock['sm'] + um_mock['icl']),
                              name='mtot_galaxy'))

    # Total stellar masses within a halo, including the satellites
    mstar_mhalo = total_stellar_mass_including_satellites(um_mock,
                                                          'mtot_galaxy')
    um_mock.add_column(Column(data=mstar_mhalo,
                              name='mstar_mhalo'))

    # Add log10(Mass)
    # Stellar mass
    um_mock.add_column(Column(data=np.log10(um_mock['sm']),
                              name='logms_gal'))
    um_mock.add_column(Column(data=np.log10(um_mock['icl']),
                              name='logms_icl'))
    um_mock.add_column(Column(data=np.log10(um_mock['mtot_galaxy']),
                              name='logms_tot'))
    um_mock.add_column(Column(data=np.log10(um_mock['mstar_mhalo']),
                              name='logms_halo'))
    # Halo mass
    um_mock.add_column(Column(data=np.log10(um_mock['mvir']),
                              name='logmh_vir'))
    um_mock.add_column(Column(data=np.log10(um_mock['mpeak']),
                              name='logmh_peak'))
    um_mock.add_column(Column(data=np.log10(um_mock['host_halo_mvir']),
                              name='logmh_host'))
    
    um_mock.rename_column('host_halo_mvir', 'mhalo_host')

    if um_min_mvir is not None:
        if use_mvir:
            um_mock_use = um_mock[um_mock['logmh_vir'] >= um_min_mvir]
        else:
            um_mock_use = um_mock[um_mock['logmh_host'] >= um_min_mvir]
    else:
        um_mock_use = um_mock

    return um_mock_use

In [31]:
um_smdpl_old_all_vagc = umPrepCatalog(um_smdpl_old_all, um_min_mvir=11.5)
um_smdpl_new_all_vagc = umPrepCatalog(um_smdpl_new_all, um_min_mvir=11.5)

print(um_smdpl_new_all_vagc.colnames)
print(len(um_smdpl_new_all_vagc))

['halo_id', 'upid', 'x', 'y', 'z', 'mvir', 'mpeak', 'sm', 'icl', 'sfr', 'halo_hostid', 'mhalo_host', 'mask_central', 'mtot_galaxy', 'mstar_mhalo', 'logms_gal', 'logms_icl', 'logms_tot', 'logms_halo', 'logmh_vir', 'logmh_peak', 'logmh_host']
11193763


In [24]:
# Save the results

# FITS table
um_smdpl_old_all_vagc.write(os.path.join(um_dir,
                                         'um_smdpl_0.7124_old_vagc_mhost_11.5.fits'),
                            format='fits', overwrite=True)
um_smdpl_new_all_vagc.write(os.path.join(um_dir,
                                         'um_smdpl_0.7124_new_vagc_mhost_11.5.fits'),
                            format='fits', overwrite=True)

# Numpy array 
np.save(os.path.join(um_dir,
                     'um_smdpl_0.7124_old_vagc_mhost_11.5.npy'),
        np.asarray(um_smdpl_old_all_vagc)
       )
np.save(os.path.join(um_dir,
                     'um_smdpl_0.7124_new_vagc_mhost_11.5.npy'),
        np.asarray(um_smdpl_new_all_vagc)
       )

## Select a sample of galaxies in massive haloes for our model

* Choose logMh_peak > 11.5 for now

In [33]:
um_smdpl_old_mvagc = um_smdpl_old_all[um_smdpl_old_all['logmh_peak'] >= 11.5]
um_smdpl_new_mvagc = um_smdpl_new_all[um_smdpl_new_all['logmh_peak'] >= 11.5]

print(len(um_smdpl_new_mvagc))

806306


In [34]:
# Save the results

# FITS table
um_smdpl_old_mvagc.write(os.path.join(um_dir,
                                      'um_smdpl_0.7124_old_vagc_mpeak_11.5.fits'),
                            format='fits', overwrite=True)
um_smdpl_new_mvagc.write(os.path.join(um_dir,
                                      'um_smdpl_0.7124_new_vagc_mpeak_11.5.fits'),
                            format='fits', overwrite=True)

# Numpy array 
np.save(os.path.join(um_dir,
                     'um_smdpl_0.7124_old_vagc_mpeak_11.5.npy'),
        np.asarray(um_smdpl_old_mvagc)
       )
np.save(os.path.join(um_dir,
                     'um_smdpl_0.7124_new_vagc_mpeak_11.5.npy'),
        np.asarray(um_smdpl_new_mvagc)
       )